In [1]:
# %run packages.ipynb

In [4]:
from pdf_files import get_fullpath_of_pdf_files

In [5]:
from pdf_documents import get_pdf_documents

In [6]:
from document_util import get_split_documents

In [7]:
from embedding_function import get_embedding_function

In [8]:
from chroma_db import create_db, get_similar_docs

In [9]:
from chain import get_chain_output

In [10]:
from match import run_text_match

In [11]:
from pdf_retrieval import save_pdfs_metadata_to_db, get_metadata_from_db, get_column_from_db

In [12]:
# path = "./data/docs/literature1" # 1 good 
path = "./data/docs/literature6" # 6 good 
# path = "./data/docs/literature9" # 9 good
# path = "./data/docs/abstracts"  # all

chunk_size=1500
chunk_overlap=100
persist_directory = 'data/db/vector/huggingface_index_all'
collection_name = 'paper_index'
collection_name_abstract = 'abstract_index'
excel_file = 'data/db/repo.xlsx'

In [13]:
embedding_function = get_embedding_function()

### The following 4 lines only need to be run once (after the pdf is loaded for the first time)

In [27]:
pdf_files = get_fullpath_of_pdf_files(path)

In [28]:
pdf_docs = get_pdf_documents(pdf_files)

In [32]:
pdf_docs

[Document(page_content="Economic growth, environmental sustainability and China mayors' promotion Zhengning Pu a, b, Jiasha Fu c, d, * a School of Economics and Management, Southeast University, 2 Si Pai Lou, Nanjing, Jiangsu Province, 210000, China b National School of Development, Southeast University, 2 Si Pai Lou, Nanjing, Jiangsu Province, 210000, China c Research Institute of Economics and Management, Southwestern University of Finance and Economics, 55 Guanghuacun Street, Chengdu, Sichuan Province, 610074, China d Survey and Research Center for China Household Finance, Southwestern University of Finance and Economics, 55 Guanghuacun Street, Chengdu, Sichuan Province, 610074, China a r t i c l e i n f o Article history: Received 28 April 2017 Received in revised form 29 September 2017 Accepted 15 October 2017 Keywords: Environmental pollution Apparent & inapparent pollutants Mayors' promotion Public pressures a b s t r a c t In this study, we follow the concept of “promotion tour

In [ ]:
split_docs = get_split_documents(pdf_docs, chunk_size, chunk_overlap)

In [ ]:
create_db(split_docs,embedding_function,collection_name,persist_directory)

### 生成标题+润色 vector DB

In [ ]:
save_pdfs_metadata_to_db(pdf_files, excel_file)

In [ ]:
abstract_docs = get_column_from_db(excel_file, 'abstract')

In [19]:
save_pdfs_metadata_to_db(pdf_files, excel_file)

get meta from LLM ./data/docs/abstracts/Crashing the party_An experimental investigation of strategic voting in primary elections.pdf
Data append to  data/db/repo.xlsx
get meta from LLM ./data/docs/abstracts/tujula2007.pdf


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID cd9398f5d9e1149be467709eb1b05846 in your message.).


Expecting property name enclosed in double quotes: line 6 column 1 (char 904)
get meta from LLM ./data/docs/abstracts/Do people care about democracy_An experiment exploring the value of voting rights.pdf
Data append to  data/db/repo.xlsx
get meta from LLM ./data/docs/abstracts/Expressive voting versus information avoidance_expenrimental evidence in the context of climate change mitigation.pdf
Data append to  data/db/repo.xlsx


In [15]:
abstract_docs = get_column_from_db(excel_file, 'abstract')

In [17]:
create_db(abstract_docs,embedding_function,collection_name_abstract,persist_directory)


Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index_all


In [14]:
# dict = get_metadata_from_db(excel_file)

## Query

### 标题+润色 embedding abstract, keywords, title 匹配相似文章，结构或是语言润色

In [25]:
my_abstract = "This paper believes that voters often follow the peak-end rule according to the theory of behavioral economics when they make voting decisions. However, at present, there is no relevant research on the relationship between economic voting and peak-end rule. This paper discusses voters’ decision process of using peak-end rule with a cross-nation panel data including 595 national elections in 70 countries from 1960 - 2020. The results show that voters do follow the peak-end rule. The average of highest annual GDP growth rate in the term and the GDP growth rate on final year before the election have positive impact on the vote share of the incumbent party. This paper also finds that voters with lower education level rely more on the peak-end rule. To solve the endogenous problems, this paper introduces an instrumental variable strategy. We introduce the global oil price and the federal fund rate as instrumental variables. "
my_title = ""
my_keywords = ""

In [26]:
similar_abstracts_docs = get_similar_docs(my_abstract, 5, embedding_function, collection_name_abstract, persist_directory)

Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index_all


In [27]:
abstracts_output = get_chain_output(my_abstract,similar_abstracts_docs)

In [29]:
abstracts_output

{'query': 'This paper believes that voters often follow the peak-end rule according to the theory of behavioral economics when they make voting decisions. However, at present, there is no relevant research on the relationship between economic voting and peak-end rule. This paper discusses voters’ decision process of using peak-end rule with a cross-nation panel data including 595 national elections in 70 countries from 1960 - 2020. The results show that voters do follow the peak-end rule. The average of highest annual GDP growth rate in the term and the GDP growth rate on final year before the election have positive impact on the vote share of the incumbent party. This paper also finds that voters with lower education level rely more on the peak-end rule. To solve the endogenous problems, this paper introduces an instrumental variable strategy. We introduce the global oil price and the federal fund rate as instrumental variables. ',
 'context': [Document(page_content='In almost all dem

### 匹配

In [10]:
query = "However, the China's cadre evaluation system is different from the voting system in the Western countries"

In [11]:
similar_docs = get_similar_docs(query, 5, embedding_function, collection_name, persist_directory)

Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index2


In [12]:
output = get_chain_output(query,similar_docs)

In [13]:
final_list = run_text_match(output,query,similar_docs, embedding_function)

No result was found, Using embedding searching strategy!!!
The 0 sentence
Sentence: Empirically, the literature linking China’s cadre eval- uation system and growth produces highly ambiguous and problematic results
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6213207978549135
========.
========.
The 1 sentence
Sentence: After the founding of the People’s Republic of China, the CCP introduced formal methods of evaluating cadres, which were formally linked to their promotion
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6769256411963147
========.
========.
The 2 sent